In [3]:
# prompt: connect only to a folder in google drive and mount it in the current directory

from google.colab import drive
import os

# Mount the specific folder in Google Drive
folder_name = "MyDrive"
mount_point = "/content/drive"

drive.mount(mount_point, force_remount = True)

# Change the current working directory to the mounted folder
os.chdir(os.path.join(mount_point, folder_name))

# Verify the current working directory
print(f"Current working directory: {os.getcwd()}")


Mounted at /content/drive
Current working directory: /content/drive/MyDrive


In [6]:
!chmod +x post-checkout

chmod: cannot access 'post-checkout': No such file or directory


In [8]:
!git clone https://github.com/refute2change/wordleGuesser.git
os.chdir("wordleGuesser")

Cloning into 'wordleGuesser'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 66 (delta 14), reused 50 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 409.67 KiB | 8.91 MiB/s, done.
fatal: premature end of pack file, 43055 bytes missing
fatal: fetch-pack: invalid index-pack output


FileNotFoundError: [Errno 2] No such file or directory: 'wordleGuesser'

In [9]:
import os

def load_data():
    # Check if the file exists
    if not os.path.exists("5letters.txt"):
        print("File not found!")
        return None

    # Load the data from the file
    with open("5letters.txt", "r") as f:
        words = f.read().splitlines()

    return words


In [10]:
import json

words = load_data()
with open("words.json", "r") as f:
    idlist = json.load(f)

In [11]:
from tqdm import tqdm
import json
from collections import Counter
import math

freqs = {}
for word in words:
    freqs[word] = Counter(word)
results = {}

def get_word_appearance(word: str, candidate: str) -> str:
    result = ['m'] * len(word)
    freq = freqs[candidate].copy()
    unchecked = []
    # First pass: Check for exact matches
    for i, (w, c) in enumerate(zip(word, candidate)):
        if w == c:
            result[i] = 'c'
            freq[w] -= 1
        elif w not in candidate:
            result[i] = 'w'
        else:
            unchecked.append(i)
    if len(unchecked) == 0:
        return "".join(result)

    # Second pass: Check for misplaced matches
    for i in unchecked:
        w = word[i]
        if freq.get(w, 0) > 0:
            freq[w] -= 1
        else:
            result[i] = 'w'

    return "".join(result)

def encode_verdict(verdict: str) -> int:
    score = 0
    for i in range(len(verdict)):
        c = verdict[-(i + 1)]
        if c == 'c':
            score += 2 * (3 ** i)
        elif c == 'm':
            score += 3 ** i
    return score

def decode_verdict(score: int, length: int) -> str:
    verdict = ['w'] * length
    for i in range(length):
        j = score % 3
        if j == 0:
            verdict[-(i + 1)] = 'w'
        elif j == 1:
            verdict[-(i + 1)] = 'm'
        elif j == 2:
            verdict[-(i + 1)] = 'c'
        score = score // 3
    return "".join(verdict)

def precompute():
    for word in tqdm(words):
        results[idlist[word]] = [0] * len(words)
        for candidate in words:
            results[idlist[word]][idlist[candidate]] = encode_verdict(get_word_appearance(word, candidate))
    with open("newprecompute.json", "w") as f:
        json.dump(results, f)

def get_entropy(word: str, candidates: list[str]) -> float:
    i = Counter(results[str(idlist[word])][idlist[candidate]] for candidate in candidates)
    s = 0 + sum(i[j]/len(candidates) * math.log(len(candidates)/i[j], 2) for j in i)
    return s

def get_prediction(guess: str, answer: str) -> str:
    return decode_verdict(results[str(idlist[guess])][idlist[answer]], len(guess))

In [12]:
#for precomputation and all-pairs verdict
precompute()

  1%|          | 68/12970 [00:04<12:52, 16.71it/s]


KeyboardInterrupt: 

In [13]:
#for loading precomputed data
with open("newprecompute.json", "r") as f:
    results = json.load(f)

In [15]:
from tqdm import tqdm
import json

class bot:
    candidates: list[str]

    def __init__(self):
        self.candidates = load_data()
        with open("precompute.json", "r") as f:
            score = json.load(f)
        with open("output.json", "w") as f:
            json.dump(score, f, indent=4)

    def get_guess(self):
        with open("output.json", "r") as f:
            score = json.load(f)
            return list(score.keys())[0]

    def narrow_down(self, guess: str, result: str):
        f = open("log_file.txt", "w")
        f.write(f"guess: {guess}, result: {result}\n")
        new_candidates = []
        for candidate in self.candidates:
            i = get_prediction(guess, candidate)
            if i == result:
                new_candidates.append(candidate)
        self.candidates = new_candidates
        print(len(self.candidates))
        score = {}
        for word in tqdm(self.candidates):
            score[word] = get_entropy(word, self.candidates)
        score = dict(sorted(score.items(), key=lambda item: item[1], reverse=True))
        json.dump(score, open("output.json", "w"), indent=4)

In [24]:
import random

class game:
    answer: str
    guesses: list[str] = []
    results: list[str] = []

    def __init__(self, length: int = 5, index: int = -1):
        if length < 3:
            raise ValueError("Length must be at least 3")
        else:
            file = open("answers/" + str(length) + "letters.txt", "r")
            words = file.read().splitlines()
            n = random.randint(0, len(words) - 1) if index == -1 else index
            self.answer = words[n]

    def show_answer(self):
        return self.answer

    def guess(self, guess: str):
        if len(self.guesses) == 6:
            return
        if len(guess) != len(self.answer):
            print("Invalid guess length")
            return
        result = get_prediction(guess, self.answer)
        self.guesses.append(guess)
        self.results.append(result)
        return result

In [26]:
#run a game
g = game(5, index = 1)
b = bot()
print(g.answer)
for i in range(6):
    guess = b.get_guess()
    result = g.guess(guess)
    print(g.answer in b.candidates)
    b.narrow_down(guess, result)
    print(f"Guess: {guess}, Result: {result}")
    if result == 'c' * len(guess):
        print("You win!")
        break

print(len(g.guesses))


abase
True
116


100%|██████████| 116/116 [00:00<00:00, 19738.70it/s]


Guess: aargh, Result: cmwww
True
3


100%|██████████| 3/3 [00:00<00:00, 1819.39it/s]


Guess: alans, Result: cwcwm
True
1


100%|██████████| 1/1 [00:00<00:00, 7626.01it/s]

Guess: abase, Result: ccccc
You win!
6
